In [1]:
'''
If you are running locally then 
1. reboot your local machine
2. create an environment called 'colab' using anaconda prompt
if you have a gpu
conda create -n colab python tensorflow-gpu 
if not 
conda create -n colab python tensorflow
3. to install jupyter notebook
conda install jupyter notebook
4. to go to the 'colab' environment
activate colab
5. change file path to locate this notebook and then type 'jupyter notebook'

If you use colab
1. save the data file in your google drive
2. goto colab and start running the code
'''

"\nIf you are running locally then \n1. reboot your local machine\n2. create an environment called 'colab' using anaconda prompt\nif you have a gpu\nconda create -n colab python tensorflow-gpu \nif not \nconda create -n colab python tensorflow\n3. to install jupyter notebook\nconda install jupyter notebook\n4. to go to the 'colab' environment\nactivate colab\n5. change file path to locate this notebook and then type 'jupyter notebook'\n\nIf you use colab\n1. save the data file in your google drive\n2. goto colab and start running the code\n"

In [2]:
#install prominent libraries with specific versions

#!pip install tensorflow==1.15.0
#!pip install keras==2.2.4-tf
#!pip install pandas==0.25.1
#!pip install sklearn==0.21.3
#!pip install matplotlib==3.2.1
#!pip install hyperas
#!pip install hyperopt

In [3]:
from __future__ import print_function                                                                                                                                                                                                                                                                                                                              # from tensorflow.contrib.rnn import *import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM,GRU, Input, Activation, Flatten, BatchNormalization, Reshape,Concatenate,Bidirectional
from keras.callbacks import ModelCheckpoint, TensorBoard, History
from keras.models import Model, load_model
from keras.layers.advanced_activations import LeakyReLU
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from matplotlib import style
from keras.utils import np_utils
from sklearn.preprocessing import*
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.utils import np_utils, to_categorical
import pandas as pd
from keras import initializers, regularizers, constraints
from keras.callbacks import ModelCheckpoint
from collections import Counter
import operator
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import itertools
import hyperas
import hyperopt
from hyperas.distributions import choice, uniform
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe, rand
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.engine.topology import Layer
from keras import backend as K

C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\f

In [4]:
#Get library versions
print("tensorflow.__version__ = ", tf.__version__)
print("keras.__version__ = ", keras.__version__)
import sklearn 
print("sklearn.__version__ = ", sklearn.__version__)
print("numpy.__version__ = ", np.__version__)
print("pandas.__version__ = ", pd.__version__)
import matplotlib
print("matplotlib.__version__ = ", matplotlib.__version__)

tensorflow.__version__ =  1.10.0
keras.__version__ =  2.1.6-tf
sklearn.__version__ =  0.21.3
numpy.__version__ =  1.17.0
pandas.__version__ =  1.0.1
matplotlib.__version__ =  3.1.3


In [5]:
#random seed to generate reproduceable results
from numpy.random import seed
seed(56)
from tensorflow import set_random_seed
set_random_seed(56)
random.seed(56)
os.environ['PYTHONHASHSEED']=str(1)

In [6]:
# create these folders if they do not exist
def build_path(dirName):
    try:
        os.makedirs(dirName)    
        print("Directory " , dirName ,  " Created ")
    except:
        print("Directory " , dirName ,  " already exists")  

In [7]:
# to check if GPU is getting used locally.....you need to see CPU as well as GPU in the output
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5218342778302130459
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7134468506
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10726184859750403017
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [8]:
def data():
  i = 7 # the label target - number of days to predict from the input date
  p = 7 #Number of days for target calculated in the data set
  batch_size=512
  CLASSES = 2
  time_steps = 7
  ticker='^GSPC'
    
  # read data
  df=pd.read_csv('../^GSPC_7_days_0_return_dtw.csv', index_col = 0, parse_dates = True)
    
  #add additional rolling mean data  
  rm_window =30
  rolling_mean = []
  for a in range(2,rm_window+1):
    df[ticker+'rm_'+str(a)] = df[ticker].rolling(window=rm_window,center=False).mean()
    rolling_mean.append(ticker+'rm_'+str(a))
    
  # create label
  targets=pd.DataFrame([])
  for j in range (1, p+1):
    targets=targets.append(df[ticker+'_{}d_target'.format(j)])
    targets=targets.append(df[ticker+'_{}d'.format(j)])
  targets=targets.T
  df=df.drop(targets.columns, axis=1)
  df=df[rm_window:-i]
  targets=targets[rm_window:-i]
  y=targets['^GSPC_{}d_target'.format(i)]

  #check for NaN and remove
  df.isna().mean().sum()
  y.isna().mean().sum()
  remove_list=[]
  for i in df.isnull().any().iteritems():
    if i[1] == True:
      remove_list.append(i[0])
  df=df.drop(remove_list, axis=1)
  df.isnull().any().mean()
 
  # add percent change
  df=df.pct_change()
  df=df.replace([np.inf, -np.inf],np.nan) 
  df.fillna(0, inplace=True)
  df.isnull().any().mean()
    
  # apply preprocessing 
  x_scaler=RobustScaler()
  x = x_scaler.fit_transform(df)
  # x_pred = x_scaler.fit_transform(x_pred)
  del df
  y=y.values
    
  # apply time steps
  def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
      v = X[i:(i + time_steps)]
      Xs.append(v)
      ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)
  x, y = create_dataset(x, y, time_steps)

  # create train and test dataset
  x_train, x_test, y_train, y_test=train_test_split(x,y, train_size=0.7, random_state=54)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  #y_train = y_train.astype('float32')
  #y_test = y_test.astype('float32')
  y_train = np_utils.to_categorical(y_train, CLASSES, dtype='float32')
  y_test = np_utils.to_categorical(y_test, CLASSES, dtype='float32')
    
  # adjustment for batch_size
  train_start = x_train.shape[0]%batch_size
  test_start = x_test.shape[0]%batch_size
  x_train = x_train[train_start:]
  y_train = y_train[train_start:]
  x_test = x_test[test_start:]
  y_test = y_test[test_start:]

  return x_train, x_test, y_train, y_test, batch_size, time_steps

In [11]:
def create_self_attention_cnn2D_bidirectional_cudnnlstm_model(x_train, x_test, y_train, y_test, batch_size, time_steps):
    
    class Attention(Layer):
        def __init__(self, step_dim,
                     W_regularizer=None, b_regularizer=None,
                     W_constraint=None, b_constraint=None,
                     bias=True, **kwargs):
            self.supports_masking = True
            self.init = initializers.get('glorot_uniform')
            self.W_regularizer = regularizers.get(W_regularizer)
            self.b_regularizer = regularizers.get(b_regularizer)
            self.W_constraint = constraints.get(W_constraint)
            self.b_constraint = constraints.get(b_constraint)
            self.bias = bias
            self.step_dim = step_dim
            self.features_dim = 0
            super(Attention, self).__init__(**kwargs)

        def build(self, input_shape):
            assert len(input_shape) == 3
            self.W = self.add_weight(shape=(input_shape[-1],),
                                     initializer=self.init,
                                     name='{}_W'.format(self.name),
                                     regularizer=self.W_regularizer,
                                     constraint=self.W_constraint)
            self.features_dim = input_shape[-1]
            if self.bias:
                self.b = self.add_weight(shape=(input_shape[1],),
                                         initializer='zero',
                                         name='{}_b'.format(self.name),
                                         regularizer=self.b_regularizer,
                                         constraint=self.b_constraint)
            else:
                self.b = None
            self.built = True

        def compute_mask(self, input, input_mask=None):
            return None

        def call(self, x, mask=None):
            features_dim = self.features_dim
            step_dim = self.step_dim
            eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))),
                            (-1, step_dim))
            if self.bias:
                eij += self.b
            eij = K.tanh(eij)
            a = K.exp(eij)
            if mask is not None:
                a *= K.cast(mask, K.floatx())
            a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
            a = K.expand_dims(a)
            weighted_input = x * a
            return K.sum(weighted_input, axis=1)

        def compute_output_shape(self, input_shape):
            return input_shape[0],  self.features_dim
   
    
   
    # define two sets of inputs(it can be the same or different)
    x_train1 = x_train.reshape(x_train.shape[0], 1, x_train.shape[1], x_train.shape[2]) 
    x_test1 = x_test.reshape(x_test.shape[0], 1, x_test.shape[1], x_test.shape[2])  
    inputA = Input(batch_shape=(batch_size, x_train1.shape[1], x_train1.shape[2], x_train1.shape[3]), name='input_1')
    inputB = Input(batch_shape=(batch_size, x_train.shape[1], x_train.shape[2]), name='input_2')
    inputC = Input(batch_shape=(batch_size, x_train.shape[1], x_train.shape[2]), name='input_3')
    
   
    
    # the first branch operates on the same or different
    x1 = Conv2D(filters = {{choice([32, 64, 128])}},name='x1', kernel_size ={{choice([2, 3, 4, 5])}},padding='same', activation='relu')(inputA)
    x2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), dim_ordering='th', name='x2')(x1)
    x3 = Conv2D(filters ={{choice([16, 32, 64])}}, name='x3', kernel_size ={{choice([2, 3, 4, 5])}},padding='same', activation='relu')(x2)
    x4 = MaxPooling2D(pool_size=(2,2), strides=(2,2), dim_ordering='th', name='x4')(x3)
    x5 = Flatten(name='x5')(x4)
    x6 = Dense({{choice([4, 6, 8, 10, 16])}},name='x6', activation='relu')(x5)
    x = Model(inputA, x6, name='x')
    
    # the second branch operates on the same or different input
    xy1 = Attention(time_steps,name='xy1Att')(inputB)
    xy2 = Dense({{choice([4, 6, 8, 10,16])}},name='xy2', activation='relu')(xy1)
    xy = Model(inputB, xy2, name='xy')
   
    
    # the third branch opreates on the same or differentinput
    y1 = Bidirectional(CuDNNLSTM(units = {{choice([32, 64, 128])}}, name='y1',return_sequences = True, stateful = True, go_backwards = True, 
                   kernel_regularizer=regularizers.l2({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}}),
                   bias_regularizer=regularizers.l1({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}})))(inputC)
    y2 = BatchNormalization(name='y2')(y1)
    y3 = Dropout({{uniform(0, 1)}},name='y3')(y2)
    y4 = Bidirectional(CuDNNLSTM(units = {{choice([16, 32, 64])}}, name='y4',return_sequences = False,stateful = True, go_backwards = True,
                   kernel_regularizer=regularizers.l2({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}}),
                   bias_regularizer=regularizers.l1({{choice([0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])}})))(y3)
    y5 = BatchNormalization(name='y5')(y4)
    y6 = Dropout({{uniform(0, 1)}},name='y6')(y5)
    y7 = Dense({{choice([4, 6, 8, 10,16])}}, name='y7', activation='relu')(y6)
    y = Model(inputC, y7,name='y')

    
    
    # combine the output of the two branches
    combined = Concatenate(name='x_xy_y_output_combined')([x.output, xy.output, y.output])
    z1 = Dropout({{uniform(0, 1)}})(combined)

    # combined outputs
    z = Dense(2, name='z', activation="softmax")(z1)

    # our model will accept the inputs of the two branches and then output buy or sell
    model = Model(inputs=[inputA, inputB, inputC], outputs=z)
    
    
    model.summary()
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer = Adam(lr={{choice([1.0, 0.1,0.01,0.001])}}))
   
    result = model.fit([x_train1, x_train, x_train], y_train,  batch_size=batch_size, validation_data=([x_test1, x_test,x_test], y_test), 
                        epochs=10, verbose=0)
    
    
    
    validation_acc = np.amax(result.history['val_acc']) 

    print('Best validation acc of epoch:', result.history['val_acc'])
    print('Train acc of epoch:', result.history['acc'])
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}


    

In [12]:
if __name__ == '__main__':
    x_train, x_test, y_train, y_test, batch_size, time_steps  = data()
    print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
    
    best_run, best_model = optim.minimize(model=create_self_attention_cnn2D_bidirectional_cudnnlstm_model, data=data ,algo=tpe.suggest, 
                                          max_evals=5,trials=Trials(), notebook_name='4.51 Self_Attention_CNN2D_bidirectional_CuDNNLSTM_muti_input',rseed=1, verbose=False)
    print("Evaluation of best performing model:")
    best_model.save("1SELF_ATTENTION_CNN_BIDIRECTIONAL_CUDNNLSTM_MULTI_INPUT_bestmodel.h5")
    print(best_model.get_config())
    x_test1 = x_test.reshape(x_test.shape[0], 1, x_test.shape[1], x_test.shape[2])  
    test_score, test_accuracy = best_model.evaluate([x_test1, x_test,x_test], y_test, batch_size=batch_size)
    print('test_score: ', test_score, ' test_accuracy: ', test_accuracy)
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

(8704, 7, 2782) (8704, 2) (3584, 7, 2782) (3584, 2)
  0%|                                                                              | 0/5 [00:00<?, ?it/s, best loss: ?]

D:\kenneth\challenge\final\temp_model.py:378: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="x2", data_format="channels_first")`

D:\kenneth\challenge\final\temp_model.py:380: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="x4", data_format="channels_first")`



__________________________________________________________________________________________________                     
Layer (type)                    Output Shape         Param #     Connected to                                          
input_3 (InputLayer)            (512, 7, 2782)       0                                                                 
__________________________________________________________________________________________________                     
input_1 (InputLayer)            (512, 1, 7, 2782)    0                                                                 
__________________________________________________________________________________________________                     
bidirectional_1 (Bidirectional) (512, 7, 64)         720896      input_3[0][0]                                         
__________________________________________________________________________________________________                     
x1 (Conv2D)                     (512, 1,

y2 (BatchNormalization)         (512, 7, 64)         256         bidirectional_3[0][0]                                 
__________________________________________________________________________________________________                     
x2 (MaxPooling2D)               (512, 1, 3, 64)      0           x1[0][0]                                              
__________________________________________________________________________________________________                     
y3 (Dropout)                    (512, 7, 64)         0           y2[0][0]                                              
__________________________________________________________________________________________________                     
x3 (Conv2D)                     (512, 1, 3, 16)      25616       x2[0][0]                                              
__________________________________________________________________________________________________                     
bidirectional_4 (Bidirectional) (512, 12

__________________________________________________________________________________________________                     
input_2 (InputLayer)            (512, 7, 2782)       0                                                                 
__________________________________________________________________________________________________                     
y5 (BatchNormalization)         (512, 128)           512         bidirectional_6[0][0]                                 
__________________________________________________________________________________________________                     
x5 (Flatten)                    (512, 16)            0           x4[0][0]                                              
__________________________________________________________________________________________________                     
xy1Att (Attention)              (512, 2782)          2789        input_2[0][0]                                         
________________________________________

x6 (Dense)                      (512, 8)             136         x5[0][0]                                              
__________________________________________________________________________________________________                     
xy2 (Dense)                     (512, 6)             16698       xy1Att[0][0]                                          
__________________________________________________________________________________________________                     
y7 (Dense)                      (512, 4)             260         y6[0][0]                                              
__________________________________________________________________________________________________                     
x_xy_y_output_combined (Concate (512, 18)            0           x6[0][0]                                              
                                                                 xy2[0][0]                                             
                                        

__________________________________________________________________________________________________                     
z (Dense)                       (512, 2)             66          dropout_5[0][0]                                       
Total params: 1,181,459                                                                                                
Trainable params: 1,181,075                                                                                            
Non-trainable params: 384                                                                                              
__________________________________________________________________________________________________                     
Best validation acc of epoch:                                                                                          
[0.5552455357142857, 0.5552455357142857, 0.5549665178571429, 0.5295758928571429, 0.48074776785714285, 0.49888392857142855, 0.48270089285714285, 0.50725446428571

{'name': 'model_3', 'layers': [{'name': 'input_3', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (512, 7, 2782), 'dtype': 'float32', 'sparse': False, 'name': 'input_3'}, 'inbound_nodes': []}, {'name': 'input_1', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (512, 1, 7, 2782), 'dtype': 'float32', 'sparse': False, 'name': 'input_1'}, 'inbound_nodes': []}, {'name': 'bidirectional_5', 'class_name': 'Bidirectional', 'config': {'name': 'bidirectional_5', 'trainable': True, 'layer': {'class_name': 'CuDNNLSTM', 'config': {'name': 'y1', 'trainable': True, 'return_sequences': True, 'return_state': False, 'go_backwards': True, 'stateful': True, 'units': 32, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': T

3584/3584 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 149us/step
test_score:  12.614515713282994  test_accuracy:  0.5552455357142857
Best performing model chosen hyper-parameters:
{'Dense': 4, 'Dense_1': 3, 'Dense_2': 3, 'Dropout': 0.1944309760994185, 'Dropout_1': 0.15353593190933112, 'Dropout_2': 0.7653610248673549, 'filters': 1, 'filters_1': 1, 'filters_2': 0, 'filters_3': 2, 'kernel_size': 0, 'kernel_size_1': 0, 'l2': 9, 'l2_1': 9, 'l2_2': 2, 'l2_3': 9, 'lr': 2}
